In [ ]:
#Imports

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import math
from transformers import CLIPProcessor, CLIPModel, AutoModel, CLIPImageProcessor, SamModel, SamProcessor

from numpy import mean
from numpy import std
from numpy import absolute

from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score, KFold
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import make_scorer,r2_score
from sklearn.svm import SVR

In [ ]:
#Loading simulated dataset

xx_bin = np.load('/content/drive/MyDrive/11_12/xx.npy')
xy_bin = np.load('/content/drive/MyDrive/11_12/xy.npy')
xz_bin = np.load('/content/drive/MyDrive/11_12/xz.npy')

y = np.load('/content/drive/MyDrive/11_12/digital_y.npy')


In [ ]:
#Loading SAM features

sam_xx_1 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_sam_1.npy')
sam_xx_2 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_sam_2.npy')
sam_xx_3 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_sam_3.npy')
sam_xx_4 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_sam_4.npy')
sam_xx_5 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_sam_5.npy')
sam_xx_5 = sam_xx_5[200:]

sam_xx = np.concatenate((sam_xx_1,sam_xx_2,sam_xx_3,sam_xx_4,sam_xx_5),axis = 0)

sam_xy_1 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_sam_1.npy')
sam_xy_2 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_sam_2.npy')
sam_xy_3 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_sam_3.npy')
sam_xy_4 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_sam_4.npy')
sam_xy_5 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_sam_5.npy')

sam_xy = np.concatenate((sam_xy_1,sam_xy_2,sam_xy_3,sam_xy_4,sam_xy_5),axis = 0)

sam_xz_1 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_sam_1.npy')
sam_xz_2 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_sam_2.npy')
sam_xz_3 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_sam_3.npy')
sam_xz_4 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_sam_4.npy')
sam_xz_5 = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_sam_5.npy')

sam_xz = np.concatenate((sam_xz_1,sam_xz_2,sam_xz_3,sam_xz_4,sam_xz_5),axis = 0)

sam_concat_features = np.concatenate((sam_xx,sam_xy,sam_xz),axis = 1)

sam_mean_features = np.mean((sam_xx,sam_xy,sam_xz),axis = 0)

print(sam_concat_features.shape)
print(sam_mean_features.shape)

In [ ]:
#CLIP

clip_vitb16_xx = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_clip_b16.npy')
clip_vitb16_xy = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_clip_b16.npy')
clip_vitb16_xz = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_clip_b16.npy')
clip_vitb32_xx = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_clip_b32.npy')
clip_vitb32_xy = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_clip_b32.npy')
clip_vitb32_xz = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_clip_b32.npy')
clip_vitl14_xx = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_clip_l14.npy')
clip_vitl14_xy = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_clip_l14.npy')
clip_vitl14_xz = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_clip_l14.npy')

clip_vitb16_concat_features = np.concatenate((clip_vitb16_xx,clip_vitb16_xy,clip_vitb16_xz),axis = 1)
clip_vitb32_concat_features = np.concatenate((clip_vitb32_xx,clip_vitb32_xy,clip_vitb32_xz),axis = 1)
clip_vitl14_concat_features = np.concatenate((clip_vitl14_xx,clip_vitl14_xy,clip_vitl14_xz),axis = 1)

clip_vitb16_mean_features = np.mean((clip_vitb16_xx,clip_vitb16_xy,clip_vitb16_xz),axis = 0)
clip_vitb32_mean_features = np.mean((clip_vitb32_xx,clip_vitb32_xy,clip_vitb32_xz),axis = 0)
clip_vitl14_mean_features = np.mean((clip_vitl14_xx,clip_vitl14_xy,clip_vitl14_xz),axis = 0)

print(clip_vitb16_concat_features.shape, clip_vitb32_concat_features.shape ,clip_vitl14_concat_features.shape)
print(clip_vitb16_mean_features.shape, clip_vitb32_mean_features.shape ,clip_vitl14_mean_features.shape)

In [ ]:
#Dino

dino_s_xx = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_dino_s.npy'))
dino_s_xy = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_dino_s.npy'))
dino_s_xz = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_dino_s.npy'))
dino_b_xx = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_dino_b.npy'))
dino_b_xy = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_dino_b.npy'))
dino_b_xz = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_dino_b.npy'))
dino_l_xx = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_dino_l.npy'))
dino_l_xy = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_dino_l.npy'))
dino_l_xz = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_dino_l.npy'))
dino_g_xx = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_dino_g.npy'))
dino_g_xy = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_dino_g.npy'))
dino_g_xz = np.squeeze(np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_dino_g.npy'))

dino_vits_concat_features = np.concatenate((dino_s_xx,dino_s_xy,dino_s_xz),axis = 1)
dino_vitb_concat_features = np.concatenate((dino_b_xx,dino_b_xy,dino_b_xz),axis = 1)
dino_vitl_concat_features = np.concatenate((dino_l_xx,dino_l_xy,dino_l_xz),axis = 1)
dino_vitg_concat_features = np.concatenate((dino_g_xx,dino_g_xy,dino_g_xz),axis = 1)

dino_vits_mean_features = np.mean((dino_s_xx,dino_s_xy,dino_s_xz),axis = 0)
dino_vitb_mean_features = np.mean((dino_b_xx,dino_b_xy,dino_b_xz),axis = 0)
dino_vitl_mean_features = np.mean((dino_l_xx,dino_l_xy,dino_l_xz),axis = 0)
dino_vitg_mean_features = np.mean((dino_g_xx,dino_g_xy,dino_g_xz),axis = 0)

print(dino_vits_concat_features.shape, dino_vitb_concat_features.shape, dino_vitl_concat_features.shape, dino_vitg_concat_features.shape)
print(dino_vits_mean_features.shape, dino_vitb_mean_features.shape, dino_vitl_mean_features.shape, dino_vitg_mean_features.shape)

In [ ]:
#NASA

nasa_xx = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_Nasa.npy')
nasa_xy = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_Nasa.npy')
nasa_xz = np.load('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_Nasa.npy')

nasa_concat_features = np.concatenate((nasa_xx,nasa_xy,nasa_xz),axis = 1)
nasa_mean_features = np.mean((nasa_xx,nasa_xy,nasa_xz),axis = 0)

print(nasa_concat_features.shape)
print(nasa_mean_features.shape)

In [ ]:
#Two Point

twopoint_xx = sio.loadmat('/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_tp_auto_periodic.mat')
twopoint_xy = sio.loadmat('/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_tp_auto_periodic.mat')
twopoint_xz = sio.loadmat('/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_tp_auto_periodic.mat')


twopoint_xx = twopoint_xx['xx_tps']
twopoint_xy = twopoint_xy['xy_tps']
twopoint_xz = twopoint_xz['xz_tps']

twopoint_xx = np.array(twopoint_xx)
twopoint_xy = np.array(twopoint_xy)
twopoint_xz = np.array(twopoint_xz)

print(twopoint_xx.shape, twopoint_xy.shape, twopoint_xz.shape)

twopoint_concat_features = np.concatenate((twopoint_xx,twopoint_xy,twopoint_xz),axis = 1)
twopoint_mean_features = np.mean((twopoint_xx,twopoint_xy,twopoint_xz),axis = 0)

print(twopoint_concat_features.shape)
print(twopoint_mean_features.shape)

In [ ]:
def MAPE(y, y_hat):
    mape = np.mean(np.abs((y - y_hat)/y))*100
    return mape

def count_terms(num_comps):
    num = num_comps + 2
    terms = math.factorial(num)/2 - 1
    return math.comb(num_comps + 2, 2)

In [ ]:
# Train / Test Splits

#CLIP
x_0_concat_clipvitb16, x_test_concat_clipvitb16, y_0_concat_clipvitb16, y_test_concat_clipvitb16 = \
train_test_split(clip_vitb16_concat_features,y,test_size=0.1,random_state=11)
x_0_concat_clipvitb32, x_test_concat_clipvitb32, y_0_concat_clipvitb32, y_test_concat_clipvitb32 = \
train_test_split(clip_vitb32_concat_features,y,test_size=0.1,random_state=11)
x_0_concat_clipvitl14, x_test_concat_clipvitl14, y_0_concat_clipvitl14, y_test_concat_clipvitl14 = \
train_test_split(clip_vitl14_concat_features,y,test_size=0.1,random_state=11)
x_0_mean_clipvitb16, x_test_mean_clipvitb16, y_0_mean_clipvitb16, y_test_mean_clipvitb16 = \
train_test_split(clip_vitb16_mean_features,y,test_size=0.1,random_state=11)
x_0_mean_clipvitb32, x_test_mean_clipvitb32, y_0_mean_clipvitb32, y_test_mean_clipvitb32 = \
train_test_split(clip_vitb32_mean_features,y,test_size=0.1,random_state=11)
x_0_mean_clipvitl14, x_test_mean_clipvitl14, y_0_mean_clipvitl14, y_test_mean_clipvitl14 = \
train_test_split(clip_vitl14_mean_features,y,test_size=0.1,random_state=11)

# #DINO
x_0_concat_dinovits, x_test_concat_dinovits, y_0_concat_dinovits, y_test_concat_dinovits = \
train_test_split(dino_vits_concat_features,y,test_size=0.1,random_state=11)
x_0_concat_dinovitb, x_test_concat_dinovitb, y_0_concat_dinovitb, y_test_concat_dinovitb = \
train_test_split(dino_vitb_concat_features,y,test_size=0.1,random_state=11)
x_0_concat_dinovitl, x_test_concat_dinovitl, y_0_concat_dinovitl, y_test_concat_dinovitl = \
train_test_split(dino_vitl_concat_features,y,test_size=0.1,random_state=11)
x_0_concat_dinovitg, x_test_concat_dinovitg, y_0_concat_dinovitg, y_test_concat_dinovitg = \
train_test_split(dino_vitg_concat_features,y,test_size=0.1,random_state=11)
x_0_mean_dinovits, x_test_mean_dinovits, y_0_mean_dinovits, y_test_mean_dinovits = \
train_test_split(dino_vits_mean_features,y,test_size=0.1,random_state=11)
x_0_mean_dinovitb, x_test_mean_dinovitb, y_0_mean_dinovitb, y_test_mean_dinovitb = \
train_test_split(dino_vitb_mean_features,y,test_size=0.1,random_state=11)
x_0_mean_dinovitl, x_test_mean_dinovitl, y_0_mean_dinovitl, y_test_mean_dinovitl = \
train_test_split(dino_vitl_mean_features,y,test_size=0.1,random_state=11)
x_0_mean_dinovitg, x_test_mean_dinovitg, y_0_mean_dinovitg, y_test_mean_dinovitg = \
train_test_split(dino_vitg_mean_features,y,test_size=0.1,random_state=11)


# #NASA
x_0_concat_nasa, x_test_concat_nasa, y_0_concat_nasa, y_test_concat_nasa = \
train_test_split(nasa_concat_features,y,test_size=0.1,random_state=11)
x_0_mean_nasa, x_test_mean_nasa, y_0_mean_nasa, y_test_mean_nasa = \
train_test_split(nasa_mean_features,y,test_size=0.1,random_state=11)

#Two Point
x_0_concat_tp, x_test_concat_tp, y_0_concat_tp, y_test_concat_tp = \
train_test_split(twopoint_concat_features,y,test_size=0.1,random_state=11)
x_0_mean_tp, x_test_mean_tp, y_0_mean_tp, y_test_mean_tp = \
train_test_split(twopoint_mean_features,y,test_size=0.1,random_state=11)

#SAM
x_0_concat_sam, x_test_concat_sam, y_0_concat_sam, y_test_concat_sam = \
train_test_split(sam_concat_features,y,test_size=0.1,random_state=11)
x_0_mean_sam, x_test_mean_sam, y_0_mean_sam, y_test_mean_sam = \
train_test_split(sam_mean_features,y,test_size=0.1,random_state=11)


In [ ]:
# Find Best PCs for 2nd order polynomial regression

def pr_pc(x_0,y_0):

    x_train,x_val,y_train,y_val = train_test_split(x_0,y_0, test_size=0.2, random_state=32)

    pca = PCA(
        svd_solver='full',
        n_components=50,
        random_state=10
    ).fit(x_train)


    x_train_red = pca.transform(x_train)
    x_val_red = pca.transform(x_val)

    poly = PolynomialFeatures(2)

    best_score = float('inf')
    best_n_components = None
    for n_components in range(1, 51):
        x_train_reduced = x_train_red[:, :n_components]
        x_val_reduced = x_val_red[:, :n_components]

        x_poly_train = poly.fit_transform(x_train_reduced)
        x_poly_val = poly.transform(x_val_reduced)

        lr = LinearRegression()
        lr.fit(x_poly_train, y_train)


        y_train_pred = lr.predict(x_poly_train)
        y_pred = lr.predict(x_poly_val)

        train_score = MAPE(y_train,y_train_pred)
        val_score = MAPE(y_val,y_pred)

        print(f"Comp: {n_components}, MAPE Train Score: {train_score:.3f}, MAPE Val Score: {val_score:.3f}")


        if val_score < best_score:
            best_score = val_score
            best_n_components = n_components

    print(f"Best Comps: {best_n_components}, MAPE Val Score: {best_score:.3f}")

    return best_score, best_n_components



In [ ]:
# Running best models for 2nd order polynomial regression

def pr(x_train,y_train,x_test,y_test,n):

    pca = PCA(
        svd_solver='full',
        n_components=n,
        random_state=10
    ).fit(x_train)


    x_train_red = pca.transform(x_train)
    x_test_red = pca.transform(x_test)

    poly = PolynomialFeatures(2)

    x_poly_train = poly.fit_transform(x_train_red)
    x_poly_test = poly.transform(x_test_red)

    lr = LinearRegression()
    lr.fit(x_poly_train, y_train)

    y_pred_train = lr.predict(x_poly_train)
    y_pred_test = lr.predict(x_poly_test)

    train_score = MAPE(y_train,y_pred_train)
    test_score = MAPE(y_test,y_pred_test)

    print(f"Train: {train_score:.3f}, Test: {test_score:.3f}")

    return train_score, test_score



In [ ]:
#CLIP

print ('Clip B16')
clipvitb16_concat_score, clipvitb16_concat_components = pr_pc(x_0_concat_clipvitb16,y_0_concat_clipvitb16)
clipvitb16_mean_score, clipvitb16_mean_components = pr_pc(x_0_mean_clipvitb16,y_0_mean_clipvitb16)

print ('Clip B32')
clipvitb32_concat_score, clipvitb32_concat_components = pr_pc(x_0_concat_clipvitb32,y_0_concat_clipvitb32)
clipvitb32_mean_score, clipvitb32_mean_components = pr_pc(x_0_mean_clipvitb32,y_0_mean_clipvitb32)

print ('Clip L14')
clipvitl14_concat_score, clipvitl14_concat_components = pr_pc(x_0_concat_clipvitl14,y_0_concat_clipvitl14)
clipvitl14_mean_score, clipvitl14_mean_components = pr_pc(x_0_mean_clipvitl14,y_0_mean_clipvitl14)

In [ ]:
# Second Order Polynomial Regression Best Results

#CLIP
print ('Clip VITB16 Model Concatenated with ', clipvitb16_concat_components, ' components')
clipvitb16_concat_train_mape, clipvitb16_concat_test_mape = pr(x_0_concat_clipvitb16, y_0_concat_clipvitb16, \
                                                                    x_test_concat_clipvitb16, y_test_concat_clipvitb16, clipvitb16_concat_components)
print ('Clip VITB16 Model Mean with ', clipvitb16_mean_components, ' components')
clipvitb16_mean_train_mape, clipvitb16_mean_test_mape= pr(x_0_mean_clipvitb16, y_0_mean_clipvitb16, \
                                                                    x_test_mean_clipvitb16, y_test_mean_clipvitb16, clipvitb16_mean_components)
print ('Clip VITB32 Model Concatenated with ', clipvitb32_concat_components, ' components')
clipvitb32_concat_train_mape, clipvitb32_concat_test_mape = pr(x_0_concat_clipvitb32, y_0_concat_clipvitb32, \
                                                                    x_test_concat_clipvitb32, y_test_concat_clipvitb32, clipvitb32_concat_components)
print ('Clip VITB32 Model Mean with ', clipvitb32_mean_components, ' components')
clipvitb32_mean_train_mape, clipvitb32_mean_test_mape = pr(x_0_mean_clipvitb32, y_0_mean_clipvitb32, \
                                                                    x_test_mean_clipvitb32, y_test_mean_clipvitb32, clipvitb32_mean_components)
print ('Clip VITL14 Model Concatenated with ', clipvitl14_concat_components, ' components')
clipvitl14_concat_train_mape, clipvitl14_concat_test_mape = pr(x_0_concat_clipvitl14, y_0_concat_clipvitl14, \
                                                                    x_test_concat_clipvitl14, y_test_concat_clipvitl14, clipvitl14_concat_components)
print ('Clip VITL14 Model Mean with ', clipvitl14_mean_components, ' components')
clipvitl14_mean_train_mape, clipvitl14_mean_test_mape = pr(x_0_mean_clipvitl14, y_0_mean_clipvitl14, \
                                                                    x_test_mean_clipvitl14, y_test_mean_clipvitl14, clipvitl14_mean_components)


print ('CLIP VITB16')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitb16_concat_pr_' + str(clipvitb16_concat_components) + '.npy'
np.save(path, [clipvitb16_concat_train_mape, clipvitb16_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitb16_mean_pr_' + str(clipvitb16_mean_components) + '.npy'
np.save(path, [clipvitb16_mean_train_mape, clipvitb16_mean_test_mape])

print ('CLIP VITB32')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitb32_concat_pr_' + str(clipvitb32_concat_components) + '.npy'
np.save(path, [clipvitb32_concat_train_mape, clipvitb32_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitb32_mean_pr_' + str(clipvitb32_mean_components) + '.npy'
np.save(path, [clipvitb32_mean_train_mape, clipvitb32_mean_test_mape])

print ('CLIP VITL14')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitl14_concat_pr_' + str(clipvitl14_concat_components) + '.npy'
np.save(path, [clipvitl14_concat_train_mape, clipvitl14_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitl14_mean_pr_' + str(clipvitl14_mean_components) + '.npy'
np.save(path, [clipvitl14_mean_train_mape, clipvitl14_mean_test_mape])



In [ ]:
#Dino
print ('Small')
dinovits_concat_score, dinovits_concat_components = pr_pc(x_0_concat_dinovits,y_0_concat_dinovits)
dinovits_mean_score, dinovits_mean_components = pr_pc(x_0_mean_dinovits,y_0_mean_dinovits)

print ('Base')
dinovitb_concat_score, dinovitb_concat_components = pr_pc(x_0_concat_dinovitb,y_0_concat_dinovitb)
dinovitb_mean_score, dinovitb_mean_components = pr_pc(x_0_mean_dinovitb,y_0_mean_dinovitb)

print ('Large')
dinovitl_concat_score, dinovitl_concat_components = pr_pc(x_0_concat_dinovitl,y_0_concat_dinovitl)
dinovitl_mean_score, dinovitl_mean_components = pr_pc(x_0_mean_dinovitl,y_0_mean_dinovitl)

print ('Giant')
dinovitg_concat_score, dinovitg_concat_components = pr_pc(x_0_concat_dinovitg,y_0_concat_dinovitg)
dinovitg_mean_score, dinovitg_mean_components = pr_pc(x_0_mean_dinovitg,y_0_mean_dinovitg)

In [ ]:
# Second Order Polynomial Regression Best Results

#DINO
print ('Dino VitS Model Concatenated with ', dinovits_concat_components, ' components')
dinovits_concat_train_mape, dinovits_concat_test_mape = pr(x_0_concat_dinovits, y_0_concat_dinovits, \
                                                                    x_test_concat_dinovits, y_test_concat_dinovits, dinovits_concat_components)
print ('Dino VitS Model Mean with ', dinovits_mean_components, ' components')
dinovits_mean_train_mape, dinovits_mean_test_mape= pr(x_0_mean_dinovits, y_0_mean_dinovits, \
                                                                    x_test_mean_dinovits, y_test_mean_dinovits, dinovits_mean_components)
print ('Dino VitB Model Concatenated with ', dinovitb_concat_components, ' components')
dinovitb_concat_train_mape, dinovitb_concat_test_mape = pr(x_0_concat_dinovitb, y_0_concat_dinovitb, \
                                                                    x_test_concat_dinovitb, y_test_concat_dinovitb, dinovitb_concat_components)
print ('Dino VitB Model Mean with ', dinovitb_mean_components, ' components')
dinovitb_mean_train_mape, dinovitb_mean_test_mape= pr(x_0_mean_dinovitb, y_0_mean_dinovitb, \
                                                                    x_test_mean_dinovitb, y_test_mean_dinovitb, dinovitb_mean_components)
print ('Dino VitL Model Concatenated with ', dinovitl_concat_components, ' components')
dinovitl_concat_train_mape, dinovitl_concat_test_mape = pr(x_0_concat_dinovitl, y_0_concat_dinovitl, \
                                                                    x_test_concat_dinovitl, y_test_concat_dinovitl, dinovitl_concat_components)
print ('Dino VitL Model Mean with ', dinovitl_mean_components, ' components')
dinovitl_mean_train_mape, dinovitl_mean_test_mape= pr(x_0_mean_dinovitl, y_0_mean_dinovitl, \
                                                                    x_test_mean_dinovitl, y_test_mean_dinovitl, dinovitl_mean_components)
print ('Dino VitG Model Concatenated with ', dinovitg_concat_components, ' components')
dinovitg_concat_train_mape, dinovitg_concat_test_mape = pr(x_0_concat_dinovitg, y_0_concat_dinovitg, \
                                                                    x_test_concat_dinovitg, y_test_concat_dinovitg, dinovitg_concat_components)
print ('Dino VitG Model Mean with ', dinovitg_mean_components, ' components')
dinovitg_mean_train_mape, dinovitg_mean_test_mape= pr(x_0_mean_dinovitg, y_0_mean_dinovitg, \
                                                                    x_test_mean_dinovitg, y_test_mean_dinovitg, dinovitg_mean_components)


print ('DINO VitS')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vits_concat_pr_' + str(dinovits_concat_components) + '.npy'
np.save(path, [dinovits_concat_train_mape, dinovits_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vits_mean_pr_' + str(dinovits_mean_components) + '.npy'
np.save(path, [dinovits_mean_train_mape, dinovits_mean_test_mape])
print ('DINO VitB')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitb_concat_pr_' + str(dinovitb_concat_components) + '.npy'
np.save(path, [dinovitb_concat_train_mape, dinovitb_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitb_mean_pr_' + str(dinovitb_mean_components) + '.npy'
np.save(path, [dinovitb_mean_train_mape, dinovitb_mean_test_mape])
print ('DINO VitL')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitl_concat_pr_' + str(dinovitl_concat_components) + '.npy'
np.save(path, [dinovitl_concat_train_mape, dinovitl_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitl_mean_pr_' + str(dinovitl_mean_components) + '.npy'
np.save(path, [dinovitl_mean_train_mape, dinovitl_mean_test_mape])
print ('DINO VitG')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitg_concat_pr_' + str(dinovitg_concat_components) + '.npy'
np.save(path, [dinovitg_concat_train_mape, dinovitg_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitg_mean_pr_' + str(dinovitg_mean_components) + '.npy'
np.save(path, [dinovitg_mean_train_mape, dinovitg_mean_test_mape])


In [ ]:
print ('SAM')
sam_concat_score, sam_concat_components = pr_pc(x_0_concat_sam,y_0_concat_sam)
sam_mean_score, sam_mean_components = pr_pc(x_0_mean_sam,y_0_mean_sam)

In [ ]:
#SAM

print ('SAM Model Concatenated with ', sam_concat_components, ' components')
sam_concat_train_mape, sam_concat_test_mape = pr(x_0_concat_sam, y_0_concat_sam, x_test_concat_sam, y_test_concat_sam, sam_concat_components)
print ('SAM Model Mean with ', sam_mean_components, ' components')
sam_mean_train_mape, sam_mean_test_mape= pr(x_0_mean_sam, y_0_mean_sam, x_test_mean_sam, y_test_mean_sam, sam_mean_components)


path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/sam_concat_pr_' + str(sam_concat_components) + '.npy'
np.save(path, [sam_concat_train_mape, sam_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/sam_mean_pr_' + str(sam_mean_components) + '.npy'
np.save(path, [sam_mean_train_mape, sam_mean_test_mape])

In [ ]:
print ('Two Point')
tp_concat_score, tp_concat_components = pr_pc(x_0_concat_tp,y_0_concat_tp)
tp_mean_score, tp_mean_components = pr_pc(x_0_mean_tp,y_0_mean_tp)


In [ ]:
# #Two Point
print ('Two Point Model Concatenated with ', tp_concat_components, ' components')
tp_concat_train_mape, tp_concat_test_mape = pr(x_0_concat_tp, y_0_concat_tp, \
                                                                    x_test_concat_tp, y_test_concat_tp, tp_concat_components)
print ('Two Point Model Mean with ', tp_mean_components, ' components')
tp_mean_train_mape, tp_mean_test_mape= pr(x_0_mean_tp, y_0_mean_tp, \
                                                                    x_test_mean_tp, y_test_mean_tp, tp_mean_components)
print ('Two Point')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/tp_concat_pr_' + str(tp_concat_components) + '.npy'
np.save(path, [tp_concat_train_mape, tp_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/tp_mean_pr_' + str(tp_mean_components) + '.npy'
np.save(path, [tp_mean_train_mape, tp_mean_test_mape])

In [ ]:
print ('NASA')
nasa_concat_score, nasa_concat_components = pr_pc(x_0_concat_nasa,y_0_concat_nasa)
nasa_mean_score, nasa_mean_components = pr_pc(x_0_mean_nasa,y_0_mean_nasa)


In [ ]:
# Second Order Polynomial Regression Best Results

#NASA
print ('CNN Model Concatenated with ', nasa_concat_components, ' components')
nasa_concat_train_mape, nasa_concat_test_mape = pr(x_0_concat_nasa, y_0_concat_nasa, \
                                                                    x_test_concat_nasa, y_test_concat_nasa, nasa_concat_components)
print ('CNN Model Mean with ', nasa_mean_components, ' components')
nasa_mean_train_mape, nasa_mean_test_mape= pr(x_0_mean_nasa, y_0_mean_nasa, \
                                                                    x_test_mean_nasa, y_test_mean_nasa, nasa_mean_components)
print ('NASA')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/nasa_concat_pr_' + str(nasa_concat_components) + '.npy'
np.save(path, [nasa_concat_train_mape, nasa_concat_test_mape])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/nasa_mean_pr_' + str(nasa_mean_components) + '.npy'
np.save(path, [nasa_mean_train_mape, nasa_mean_test_mape])


In [ ]:
# Find Best PCs for linear regression

def lr_pc(x_0,y_0):

    x_train,x_val,y_train,y_val = train_test_split(x_0,y_0, test_size=0.2, random_state=32)

    pca = PCA(
        svd_solver='full',
        n_components=100,
        random_state=10
    ).fit(x_train)


    x_train_red = pca.transform(x_train)
    x_val_red = pca.transform(x_val)

    best_score = float('inf')
    best_n_components = None
    for n_components in range(1, 51):
        x_train_reduced = x_train_red[:, :n_components]
        x_val_reduced = x_val_red[:, :n_components]

        lr = LinearRegression()
        lr.fit(x_train_reduced, y_train)


        y_train_pred = lr.predict(x_train_reduced)
        y_pred = lr.predict(x_val_reduced)

        train_score = MAPE(y_train,y_train_pred)
        val_score = MAPE(y_val,y_pred)

        print(f"Comp: {n_components}, MAPE Train Score: {train_score:.3f}, MAPE Val Score: {val_score:.3f}")


        if val_score < best_score:
            best_score = val_score
            best_n_components = n_components

    print(f"Best Comps: {best_n_components}, MAPE Val Score: {best_score:.3f}")

    return best_score, best_n_components



In [ ]:
# Running best models for linear regression

def lr(x_train,y_train,x_test,y_test,n):

    pca = PCA(
        svd_solver='full',
        n_components=n,
        random_state=10
    ).fit(x_train)


    x_train_red = pca.transform(x_train)
    x_test_red = pca.transform(x_test)

    lr = LinearRegression()
    lr.fit(x_train_red, y_train)

    y_pred_train = lr.predict(x_train_red)
    y_pred_test = lr.predict(x_test_red)

    train_score = MAPE(y_train,y_pred_train)
    test_score = MAPE(y_test,y_pred_test)

    print(f"Train: {train_score:.3f}, Test: {test_score:.3f}")

    return train_score, test_score



In [ ]:
#CLIP

print ('Clip B16')
clipvitb16_concat_lr, clipvitb16_concat_lr_components = lr_pc(x_0_concat_clipvitb16,y_0_concat_clipvitb16)
clipvitb16_mean_lr, clipvitb16_mean_lr_components = lr_pc(x_0_mean_clipvitb16,y_0_mean_clipvitb16)

print ('Clip B32')
clipvitb32_concat_lr, clipvitb32_concat_lr_components = lr_pc(x_0_concat_clipvitb32,y_0_concat_clipvitb32)
clipvitb32_mean_lr, clipvitb32_mean_lr_components = lr_pc(x_0_mean_clipvitb32,y_0_mean_clipvitb32)

print ('Clip L14')
clipvitl14_concat_lr, clipvitl14_concat_lr_components = lr_pc(x_0_concat_clipvitl14,y_0_concat_clipvitl14)
clipvitl14_mean_lr, clipvitl14_mean_lr_components = lr_pc(x_0_mean_clipvitl14,y_0_mean_clipvitl14)

In [ ]:
# Linear Regression Best Results

#CLIP
print ('Clip VITB16 Model Concatenated with ', clipvitb16_concat_lr_components, ' components')
clipvitb16_concat_train_lr, clipvitb16_concat_test_lr = lr(x_0_concat_clipvitb16, y_0_concat_clipvitb16, \
                                                                    x_test_concat_clipvitb16, y_test_concat_clipvitb16, clipvitb16_concat_lr_components)
print ('Clip VITB16 Model Mean with ', clipvitb16_mean_lr_components, ' components')
clipvitb16_mean_train_lr, clipvitb16_mean_test_lr= lr(x_0_mean_clipvitb16, y_0_mean_clipvitb16, \
                                                                    x_test_mean_clipvitb16, y_test_mean_clipvitb16, clipvitb16_mean_lr_components)
print ('Clip VITB32 Model Concatenated with ', clipvitb32_concat_lr_components, ' components')
clipvitb32_concat_train_lr, clipvitb32_concat_test_lr = lr(x_0_concat_clipvitb32, y_0_concat_clipvitb32, \
                                                                    x_test_concat_clipvitb32, y_test_concat_clipvitb32, clipvitb32_concat_lr_components)
print ('Clip VITB32 Model Mean with ', clipvitb32_mean_lr_components, ' components')
clipvitb32_mean_train_lr, clipvitb32_mean_test_lr = lr(x_0_mean_clipvitb32, y_0_mean_clipvitb32, \
                                                                    x_test_mean_clipvitb32, y_test_mean_clipvitb32, clipvitb32_mean_lr_components)
print ('Clip VITL14 Model Concatenated with ', clipvitl14_concat_lr_components, ' components')
clipvitl14_concat_train_lr, clipvitl14_concat_test_lr = lr(x_0_concat_clipvitl14, y_0_concat_clipvitl14, \
                                                                    x_test_concat_clipvitl14, y_test_concat_clipvitl14, clipvitl14_concat_lr_components)
print ('Clip VITL14 Model Mean with ', clipvitl14_mean_lr_components, ' components')
clipvitl14_mean_train_lr, clipvitl14_mean_test_lr = lr(x_0_mean_clipvitl14, y_0_mean_clipvitl14, \
                                                                    x_test_mean_clipvitl14, y_test_mean_clipvitl14, clipvitl14_mean_lr_components)


print ('CLIP VITB16')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitb16_concat_lr_' + str(clipvitb16_concat_lr_components) + '.npy'
np.save(path, [clipvitb16_concat_train_lr, clipvitb16_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitb16_mean_lr_' + str(clipvitb16_mean_lr_components) + '.npy'
np.save(path, [clipvitb16_mean_train_lr, clipvitb16_mean_test_lr])

print ('CLIP VITB32')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitb32_concat_lr_' + str(clipvitb32_concat_lr_components) + '.npy'
np.save(path, [clipvitb32_concat_train_lr, clipvitb32_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitb32_mean_lr_' + str(clipvitb32_mean_lr_components) + '.npy'
np.save(path, [clipvitb32_mean_train_lr, clipvitb32_mean_test_lr])

print ('CLIP VITL14')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitl14_concat_lr_' + str(clipvitl14_concat_lr_components) + '.npy'
np.save(path, [clipvitl14_concat_train_lr, clipvitl14_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/clip_vitl14_mean_lr_' + str(clipvitl14_mean_lr_components) + '.npy'
np.save(path, [clipvitl14_mean_train_lr, clipvitl14_mean_test_lr])


In [ ]:
#Dino
print ('Small')
dinovits_concat_lr, dinovits_concat_lr_components = lr_pc(x_0_concat_dinovits,y_0_concat_dinovits)
dinovits_mean_lr, dinovits_mean_lr_components = lr_pc(x_0_mean_dinovits,y_0_mean_dinovits)

print ('Base')
dinovitb_concat_lr, dinovitb_concat_lr_components = lr_pc(x_0_concat_dinovitb,y_0_concat_dinovitb)
dinovitb_mean_lr, dinovitb_mean_lr_components = lr_pc(x_0_mean_dinovitb,y_0_mean_dinovitb)

print ('Large')
dinovitl_concat_lr, dinovitl_concat_lr_components = lr_pc(x_0_concat_dinovitl,y_0_concat_dinovitl)
dinovitl_mean_lr, dinovitl_mean_lr_components = lr_pc(x_0_mean_dinovitl,y_0_mean_dinovitl)

print ('Giant')
dinovitg_concat_lr, dinovitg_concat_lr_components = lr_pc(x_0_concat_dinovitg,y_0_concat_dinovitg)
dinovitg_mean_lr, dinovitg_mean_lr_components = lr_pc(x_0_mean_dinovitg,y_0_mean_dinovitg)

In [ ]:
# Second Order Polynomial Regression Best Results

#DINO
print ('Dino VitS Model Concatenated with ', dinovits_concat_lr_components, ' components')
dinovits_concat_train_lr, dinovits_concat_test_lr = lr(x_0_concat_dinovits, y_0_concat_dinovits, \
                                                                    x_test_concat_dinovits, y_test_concat_dinovits, dinovits_concat_lr_components)
print ('Dino VitS Model Mean with ', dinovits_mean_lr_components, ' components')
dinovits_mean_train_lr, dinovits_mean_test_lr = lr(x_0_mean_dinovits, y_0_mean_dinovits, \
                                                                    x_test_mean_dinovits, y_test_mean_dinovits, dinovits_mean_lr_components)
print ('Dino VitB Model Concatenated with ', dinovitb_concat_lr_components, ' components')
dinovitb_concat_train_lr, dinovitb_concat_test_lr = lr(x_0_concat_dinovitb, y_0_concat_dinovitb, \
                                                                    x_test_concat_dinovitb, y_test_concat_dinovitb, dinovitb_concat_lr_components)
print ('Dino VitB Model Mean with ', dinovitb_mean_lr_components, ' components')
dinovitb_mean_train_lr, dinovitb_mean_test_lr = lr(x_0_mean_dinovitb, y_0_mean_dinovitb, \
                                                                    x_test_mean_dinovitb, y_test_mean_dinovitb, dinovitb_mean_lr_components)
print ('Dino VitL Model Concatenated with ', dinovitl_concat_lr_components, ' components')
dinovitl_concat_train_lr, dinovitl_concat_test_lr = lr(x_0_concat_dinovitl, y_0_concat_dinovitl, \
                                                                    x_test_concat_dinovitl, y_test_concat_dinovitl, dinovitl_concat_lr_components)
print ('Dino VitL Model Mean with ', dinovitl_mean_lr_components, ' components')
dinovitl_mean_train_lr, dinovitl_mean_test_lr = lr(x_0_mean_dinovitl, y_0_mean_dinovitl, \
                                                                    x_test_mean_dinovitl, y_test_mean_dinovitl, dinovitl_mean_lr_components)
print ('Dino VitG Model Concatenated with ', dinovitg_concat_lr_components, ' components')
dinovitg_concat_train_lr, dinovitg_concat_test_lr = lr(x_0_concat_dinovitg, y_0_concat_dinovitg, \
                                                                    x_test_concat_dinovitg, y_test_concat_dinovitg, dinovitg_concat_lr_components)
print ('Dino VitG Model Mean with ', dinovitg_mean_lr_components, ' components')
dinovitg_mean_train_lr, dinovitg_mean_test_lr = lr(x_0_mean_dinovitg, y_0_mean_dinovitg, \
                                                                    x_test_mean_dinovitg, y_test_mean_dinovitg, dinovitg_mean_lr_components)


print ('DINO VitS')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vits_concat_lr_' + str(dinovits_concat_lr_components) + '.npy'
np.save(path, [dinovits_concat_train_lr, dinovits_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vits_mean_lr_' + str(dinovits_mean_lr_components) + '.npy'
np.save(path, [dinovits_mean_train_lr, dinovits_mean_test_lr])
print ('DINO VitB')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitb_concat_lr_' + str(dinovitb_concat_lr_components) + '.npy'
np.save(path, [dinovitb_concat_train_lr, dinovitb_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitb_mean_lr_' + str(dinovitb_mean_lr_components) + '.npy'
np.save(path, [dinovitb_mean_train_lr, dinovitb_mean_test_lr])
print ('DINO VitL')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitl_concat_lr_' + str(dinovitl_concat_lr_components) + '.npy'
np.save(path, [dinovitl_concat_train_lr, dinovitl_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitl_mean_lr_' + str(dinovitl_mean_lr_components) + '.npy'
np.save(path, [dinovitl_mean_train_lr, dinovitl_mean_test_lr])
print ('DINO VitG')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitg_concat_lr_' + str(dinovitg_concat_lr_components) + '.npy'
np.save(path, [dinovitg_concat_train_lr, dinovitg_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/dino_vitg_mean_lr_' + str(dinovitg_mean_lr_components) + '.npy'
np.save(path, [dinovitg_mean_train_lr, dinovitg_mean_test_lr])


In [ ]:
print ('NASA')
nasa_concat_lr, nasa_concat_lr_components = lr_pc(x_0_concat_nasa,y_0_concat_nasa)
nasa_mean_lr, nasa_mean_lr_components = lr_pc(x_0_mean_nasa,y_0_mean_nasa)

print ('Two Point')
tp_concat_lr, tp_concat_lr_components = lr_pc(x_0_concat_tp,y_0_concat_tp)
tp_mean_lr, tp_mean_lr_components = lr_pc(x_0_mean_tp,y_0_mean_tp)

In [ ]:

# #Two Point
print ('Two Point Model Concatenated with ', tp_concat_lr_components, ' components')
tp_concat_train_lr, tp_concat_test_lr = lr(x_0_concat_tp, y_0_concat_tp, \
                                                                    x_test_concat_tp, y_test_concat_tp, tp_concat_lr_components)
print ('Two Point Model Mean with ', tp_mean_components, ' components')
tp_mean_train_lr, tp_mean_test_lr= lr(x_0_mean_tp, y_0_mean_tp, \
                                                                    x_test_mean_tp, y_test_mean_tp, tp_mean_lr_components)
print ('Two Point')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/tp_concat_lr_' + str(tp_concat_lr_components) + '.npy'
np.save(path, [tp_concat_train_lr, tp_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/tp_mean_lr_' + str(tp_mean_lr_components) + '.npy'
np.save(path, [tp_mean_train_lr, tp_mean_test_lr])

In [ ]:
# Linear Regression Best Results

#NASA
print ('CNN Model Concatenated with ', nasa_concat_lr_components, ' components')
nasa_concat_train_lr, nasa_concat_test_lr = lr(x_0_concat_nasa, y_0_concat_nasa, \
                                                                    x_test_concat_nasa, y_test_concat_nasa, nasa_concat_lr_components)
print ('CNN Model Mean with ', nasa_mean_lr_components, ' components')
nasa_mean_train_lr, nasa_mean_test_lr = lr(x_0_mean_nasa, y_0_mean_nasa, \
                                                                    x_test_mean_nasa, y_test_mean_nasa, nasa_mean_lr_components)
print ('NASA')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/nasa_concat_lr_' + str(nasa_concat_lr_components) + '.npy'
np.save(path, [nasa_concat_train_lr, nasa_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/nasa_mean_lr_' + str(nasa_mean_lr_components) + '.npy'
np.save(path, [nasa_mean_train_lr, nasa_mean_test_lr])


In [ ]:
print ('SAM')
sam_concat_lr,sam_concat_lr_components = lr_pc(x_0_concat_sam,y_0_concat_sam)
sam_mean_lr, sam_mean_lr_components = lr_pc(x_0_mean_sam,y_0_mean_sam)

In [ ]:
# Linear Regression Best Results

#SAM
print ('SAM Model Concatenated with ', sam_concat_lr_components, ' components')
sam_concat_train_lr, sam_concat_test_lr = lr(x_0_concat_sam, y_0_concat_sam, \
                                                                    x_test_concat_sam, y_test_concat_sam, sam_concat_lr_components)
print ('SAM Model Mean with ', sam_mean_lr_components, ' components')
sam_mean_train_lr, sam_mean_test_lr = lr(x_0_mean_sam, y_0_mean_sam, \
                                                                    x_test_mean_sam, y_test_mean_sam, sam_mean_lr_components)
print ('SAM')
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/sam_concat_lr_' + str(sam_concat_lr_components) + '.npy'
np.save(path, [sam_concat_train_lr, sam_concat_test_lr])
path = '/content/drive/MyDrive/11_12/12_10_Digital_Results/sam_mean_lr_' + str(sam_mean_lr_components) + '.npy'
np.save(path, [sam_mean_train_lr, sam_mean_test_lr])

In [ ]:
def screeplot(X,y,n=10,order=3,scree=True,name=""):

    pca = PCA(
        svd_solver='full',
        n_components=30,
        random_state=10
    ).fit(X)

    #Calculate cumulative explained variance
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

    # Find the number of components explaining 90% variance
    n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1

    print(n_components_90)
    components = np.linspace(1,30,30)


    xs = [1,5,10,15,20,25,30]

    print (components)
    if scree:
        fig = plt.figure()
        plt.stem(components + 0.2, np.cumsum(pca.explained_variance_ratio_),linefmt = '#52B5F9', label = "Cumulative Explained Variance")# linefmt=f'{color}-', markerfmt=f'{color}o', basefmt='k')  # 'k' is for black color for the baseline
        #plt.plot(np.cumsum(pca.explained_variance_ratio_))
        plt.stem(components - 0.2, pca.explained_variance_ratio_, linefmt = '#EA334B', label = "Explained Variance")
        plt.xlabel('Number of Components', fontsize = 18)
        plt.ylabel('Explained Variance', fontsize = 18)
        plt.ylim(0,1)
        plt.xlim(0.5,30.5)
        plt.xticks(xs)
        #plt.title('Scree Plot')
        #plt.legend()
        #plt.grid()
    #plt.save('Scree.pdf')
        fig.savefig(name, bbox_inches='tight')
    plt.show()

    return

In [ ]:
def pcaplot(X,y,name):

    pca = PCA(
        svd_solver='full',
        n_components=2,
        random_state=10
    ).fit(X)

    x_tran = pca.transform(X)
    fig = plt.figure()
    plt.scatter(x_tran[:,0], x_tran[:,1], c = frac, cmap = 'YlGnBu', vmin=0, vmax=1) #'ocean_r'
    cbar = plt.colorbar()
    #cbar.set_clim(0, 1)
    fig.savefig(name, bbox_inches='tight')
    plt.show()


In [ ]:
screeplot(clip_vitb16_concat_features,y, name = "Scree_clip.pdf")

In [ ]:
screeplot(twopoint_concat_features, y, name = "Scree_tp.pdf")

In [ ]:
frac = []

for i in range(0,len(xx_bin)):
  count_ones_xx = np.count_nonzero(xx_bin[i] == 1)
  count_ones_xy = np.count_nonzero(xy_bin[i] == 1)
  count_ones_xz = np.count_nonzero(xz_bin[i] == 1)
  count_zeros_xx = np.count_nonzero(xx_bin[i] == 0)
  count_zeros_xy = np.count_nonzero(xy_bin[i] == 0)
  count_zeros_xz = np.count_nonzero(xz_bin[i] == 0)
  tot_count_ones = count_ones_xx + count_ones_xy + count_ones_xz
  tot_count_zeros = count_zeros_xx + count_zeros_xy + count_zeros_xz
  print (tot_count_ones/(tot_count_ones + tot_count_zeros))
  frac.append(tot_count_ones/(tot_count_ones + tot_count_zeros))

In [ ]:
pcaplot(clip_vitb16_concat_features,y, name = "PCAPlot_clip.pdf")

In [ ]:
pcaplot(twopoint_concat_features,y, name = "PCAPlot_tp.pdf")

In [13]:
def parity(x_train, y_train, x,y,num_comp, scale = False):

    pca = PCA(
        svd_solver='full',
        n_components=num_comp,
        random_state=10
    ).fit(x_train)


    x_train_red = pca.transform(x_train)
    x_test_red = pca.transform(x)

    poly = PolynomialFeatures(2)

    x_poly_train = poly.fit_transform(x_train_red)
    x_poly_test = poly.transform(x_test_red)

    lr = LinearRegression()
    lr.fit(x_poly_train, y_train)

    y_pred_train = lr.predict(x_poly_train)
    y_pred_test = lr.predict(x_poly_test)

    train_score = MAPE(y_train,y_pred_train)
    test_score = MAPE(y,y_pred_test)


    # Create a square figure
    fig = plt.figure(figsize=(6, 6))  # 6x6 inches square figure

    # Plot data
    x_vals = range(-10,115)
    y_vals = range(-10,115)
    plt.plot(x_vals,y_vals, color = 'black')
    plt.scatter(y_train,y_pred_train,color="#C1BCB6", zorder = 2)
    plt.scatter(y,y_pred_test,color="#FF206E", zorder = 3) #"#00E632")

    plt.xlabel('True hardness HV', fontsize = 18)
    plt.ylabel('Predicted hardness HV', fontsize = 18)
    plt.xlim(-10,115)
    plt.ylim(-10,115)
    #plt.ylim(1,7)#200,600)
    # Set equal scaling by setting aspect ratio to 'equal'
    plt.gca().set_aspect('equal', adjustable='box')

    # Save the figure as a PDF file
    name = str(num_comp) + '.pdf'
    plt.savefig(name, format='pdf')

    # Show the plot
    plt.show()

    print(f"Train: {train_score:.3f}, Test: {test_score:.3f}")

    return train_score, test_score

In [ ]:
parity(x_0_concat_clipvitb16, y_0_concat_clipvitb16,  x_test_concat_clipvitb16, y_test_concat_clipvitb16 ,39)

In [ ]:
parity(x_0_concat_tp, y_0_concat_tp,  x_test_concat_tp, y_test_concat_tp ,24)
